Reference
    
https://keras.io/examples/timeseries/timeseries_anomaly_detection/

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Define-Functions" data-toc-modified-id="Define-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define Functions</a></span><ul class="toc-item"><li><span><a href="#Visualization" data-toc-modified-id="Visualization-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Visualization</a></span><ul class="toc-item"><li><span><a href="#def-plotTimeSeries()" data-toc-modified-id="def-plotTimeSeries()-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span><code>def plotTimeSeries()</code></a></span></li></ul></li><li><span><a href="#Find-window-size" data-toc-modified-id="Find-window-size-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Find window size</a></span><ul class="toc-item"><li><span><a href="#def-findDominantFreq():---Find-the-freq-with-strongest-strength-in-Periodogram" data-toc-modified-id="def-findDominantFreq():---Find-the-freq-with-strongest-strength-in-Periodogram-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span><code>def findDominantFreq():</code> - Find the freq with strongest strength in Periodogram</a></span></li><li><span><a href="#def-findAutoPeriod()---Find-the-period-of-the-time-series-data" data-toc-modified-id="def-findAutoPeriod()---Find-the-period-of-the-time-series-data-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span><code>def findAutoPeriod()</code> - Find the period of the time series data</a></span></li></ul></li><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Prepare Data</a></span><ul class="toc-item"><li><span><a href="#def-normalize(df):" data-toc-modified-id="def-normalize(df):-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span><code>def normalize(df):</code></a></span></li></ul></li><li><span><a href="#Discretize-the-Time-series-data-by-sliding-window" data-toc-modified-id="Discretize-the-Time-series-data-by-sliding-window-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Discretize the Time series data by sliding window</a></span><ul class="toc-item"><li><span><a href="#def-createSequences(values,-time_steps=100):" data-toc-modified-id="def-createSequences(values,-time_steps=100):-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span><code>def createSequences(values, time_steps=100):</code></a></span></li></ul></li><li><span><a href="#Build,-Train-&amp;-Save-model" data-toc-modified-id="Build,-Train-&amp;-Save-model-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Build, Train &amp; Save model</a></span><ul class="toc-item"><li><span><a href="#def-buildAutoencoder(train_df,-model_name,-folder):" data-toc-modified-id="def-buildAutoencoder(train_df,-model_name,-folder):-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span><code>def buildAutoencoder(train_df, model_name, folder):</code></a></span></li><li><span><a href="#def-saveAutoencoder(model_name,-folder):" data-toc-modified-id="def-saveAutoencoder(model_name,-folder):-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span><code>def saveAutoencoder(model_name, folder):</code></a></span></li><li><span><a href="#def-trainAutoencoder(x_train,-model):" data-toc-modified-id="def-trainAutoencoder(x_train,-model):-2.5.3"><span class="toc-item-num">2.5.3&nbsp;&nbsp;</span><code>def trainAutoencoder(x_train, model):</code></a></span></li><li><span><a href="#def-plotLoss(history):" data-toc-modified-id="def-plotLoss(history):-2.5.4"><span class="toc-item-num">2.5.4&nbsp;&nbsp;</span><code>def plotLoss(history):</code></a></span></li></ul></li><li><span><a href="#Save-plot" data-toc-modified-id="Save-plot-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Save plot</a></span><ul class="toc-item"><li><span><a href="#def-savePlot():" data-toc-modified-id="def-savePlot():-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span><code>def savePlot():</code></a></span></li></ul></li><li><span><a href="#Detecting-anomalies" data-toc-modified-id="Detecting-anomalies-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Detecting anomalies</a></span><ul class="toc-item"><li><span><a href="#def-detectAnomalies(x_train,-x_test):" data-toc-modified-id="def-detectAnomalies(x_train,-x_test):-2.7.1"><span class="toc-item-num">2.7.1&nbsp;&nbsp;</span><code>def detectAnomalies(x_train, x_test):</code></a></span></li><li><span><a href="#def-findAnomaliesRange(anomalies,-_train):" data-toc-modified-id="def-findAnomaliesRange(anomalies,-_train):-2.7.2"><span class="toc-item-num">2.7.2&nbsp;&nbsp;</span><code>def findAnomaliesRange(anomalies, _train):</code></a></span></li><li><span><a href="#def-plotAnomalies(df_test,-anomalies):" data-toc-modified-id="def-plotAnomalies(df_test,-anomalies):-2.7.3"><span class="toc-item-num">2.7.3&nbsp;&nbsp;</span><code>def plotAnomalies(df_test, anomalies):</code></a></span></li></ul></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Main</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Build,-Train-and-Test-Autoencoder-for-all-data" data-toc-modified-id="Build,-Train-and-Test-Autoencoder-for-all-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Build, Train and Test Autoencoder for all data</a></span></li></ul></li></ul></div>

# Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from typing import Iterable

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import scipy.signal as signal
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf

from tensorflow import keras
from tensorflow.keras import layers

from loadData import DataLoader


# Define Functions

## Visualization

### `def plotTimeSeries()`

In [2]:
def plotTimeSeries(df, figsize_n: int = 80, figsize_m: int = 6, title='', title_fontsize=60):
    fig, ax = plt.subplots(figsize=(figsize_n, figsize_m))
    df.plot(legend=False, ax=ax)
    plt.title(title, fontsize=title_fontsize)
    plt.show()

## Find window size

http://www.l3s.de/~anand/tir14/lectures/ws14-tir-foundations-2.pdf

Steps to identify `Auto-Period` (i.e. values of x-axis time `t` that the time series signal repeats itself):
1. Find the dominant frequency by the highest "power at that freq" using "Power Spectral Density (PSD) Estimation" in the Periodogram
    p.s. Periodogram encodes the strength at a given frequency
2. Use candidate periods from Periodogram, and validate false alarms using Auto-correlation

<img src="../img/periodogram+autocorrelation.png">

After tested in `00-exploratory-data-analysis-(kdd-cup-2021).ipynb`, the `window size` should be smaller OR larger than the `auto-period`, and should not be same as `auto-period`.

### `def findDominantFreq():` - Find the freq with strongest strength in Periodogram

In [3]:
def findDominantFreq(x) -> Iterable[Iterable]:
    '''
    Parameters
    ----------
    x : array_like
        Time series of measurement values

    Return
    ----------
    f: array_like
       Array of sample frequencies.
    Pxx_den: array_like
             Power spectral density or power spectrum of `x`.
    '''
#     freq_ls = [] #
#     Pxx_den_ls = [] #Power spectral density or power spectrum of `x`.

    f, Pxx_den = signal.periodogram(x)
#     freq_ls.append(f[1:])
#     Pxx_den_ls.append(p[1:])

    return f, Pxx_den

### `def findAutoPeriod()` - Find the period of the time series data

In [23]:
def findAutoPeriod(x):
    '''Find the period of the time series data by (1) Periodogram and (2) Auto-correlation

    Steps
    ---
    1. Find the dominant frequency by the highest "power at that freq" using "Power Spectral Density (PSD) Estimation" in the Periodogram
    2. Use candidate periods from Periodogram, and validate false alarms using Auto-correlation

    P.S. Periodogram encodes the strength at a given frequency

    Parameters
    ----------
    x : array_like
        Time series of measurement values
    '''
    freq, Pxx_den = findDominantFreq(x)

    periodicity_df = pd.DataFrame(
        {'freq': freq[1:], 'PSD': Pxx_den[1:], 'Period': 1/freq[1:]}) #skip the 1st element because it is 0.0

    color = ['r', 'b', 'g', 'k', 'c']

    df = periodicity_df

    # Period hints capture from periodogram
    hints = list(round(df[(df["Period"] >= 10)].sort_values(
        'PSD', ascending=False)[:5]['Period']))

    print(hints)
#     plot_acf(x, lags=1000)
#     plt.show()

    acf_record = acf(x, nlags=max(hints), fft=True)

    # plt.plot(x,marker="|")
    # for k in range(len(hints[j])):
    #    plt.axvline(x=hints[j][k],color=color[k])
    # plt.show()

    temp = acf_record[[int(h-1) for h in hints]]
    autoperiod = hints[np.array(temp).argmax()]

    return autoperiod

## Prepare Data

### `def normalize(df):`

In [5]:
def normalize(df: pd.DataFrame,
              df2: pd.DataFrame = None
              ) -> pd.DataFrame:
    '''Normalize data by the mean and std.
    Return the normalized data value
    '''

    msg = 'xxx'
    if df2 is None:
        df2 = df
        msg = 'same'
    else:
        msg = 'other'
    print(f'> Normalizing by {msg} `mean` `std`.')

    _mean = df2.mean()
    _std = df2.std()
    df_value = (df - _mean) / _std

#     print("Number of samples:", len(df_value))

#     display(df)

#     display(df_value)

#     display(df.describe())

#     df_value.describe()

    return df_value

## Discretize the Time series data by sliding window

### `def createSequences(values, time_steps=100):`

In [6]:
def createSequences(values, time_steps: int = 100):
    '''Generated training sequences for use in the model. Return the discretized time series data.
    
    Parameters
    ----------
    values : array_like
             Time series of measurement values
    '''
    output = []
    for i in range(int(len(values) - time_steps + 1)):
        output.append(values[i: (i + time_steps)])
    return np.stack(output)

## Build, Train & Save model

### `def buildAutoencoder(train_df, model_name, folder):`

In [7]:
def buildAutoencoder(shape: tuple,
                     learning_rate=0.01,
                    ):
    print('> buildAutoencoder(), shape=', shape)
    model = keras.Sequential(
        [
            layers.Input(shape=shape),
            layers.Conv1D(filters=16, kernel_size=4, padding="same", strides=2, activation="relu"),
            layers.Dropout(rate=0.2),
            layers.Conv1D(filters=8, kernel_size=4, padding="same", strides=2, activation="relu"),
            layers.Conv1DTranspose(filters=8, kernel_size=4, padding="same", strides=2, activation="relu"),
            layers.Dropout(rate=0.2),
            layers.Conv1DTranspose(filters=16, kernel_size=4, padding="same", strides=2, activation="relu"),
            layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
#             layers.Cropping1D(cropping=(1,1)) # this is the added step
            layers.Reshape(shape)

        ]
    )
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
#     model.summary()

    return model

### `def saveAutoencoder(model_name, folder):`

In [8]:
def saveAutoencoder(model,
                    model_name: str = '',
                    folder: str = 'models/autoencoder',
                   ):
    if not os.path.exists(folder):
        os.makedirs(folder)
    path = f'{folder}/{model_name+".h5"}'
    model.save(path)
    print(f'> Saved `{path}` !')

### `def trainAutoencoder(x_train, model):`


In [9]:
def trainAutoencoder(discretized_X, model, batch_size=32):
    history = model.fit(
        discretized_X,  # x_train_padded_zero,
        discretized_X,  # x_train_padded_zero,
        epochs=50,
        batch_size=batch_size,
        validation_split=0.1,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=5, mode="min")
        ],
        verbose=2,
    )
#     plotLoss(history)
    return model, history

### `def plotLoss(history):`

In [10]:
def plotLoss(history):
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.legend()
#     plt.show()

## Save plot

### `def savePlot():`

## Detecting anomalies

### `def detectAnomalies(x_train, x_test):`

https://towardsdatascience.com/master-the-art-of-subplots-in-python-45f7884f3d2e

In [11]:
class detectAnomalies():

    def __init__(self,
                 model):
        self.x_train = []
        self.x_test = []
        self.x_train_pred = []
        self.train_mae_loss = []
        self.threshold = None
        self.x_test_pred = []
        self.test_mae_loss = []
        self.anomalies = []
        self.anomalies_range = []
        self.anomalies_group = []

    def detect(self, x_train, x_test):
        '''Find and return anomaly'''
        print("> Test input shape: ", x_test.shape)

        # Get train MAE loss.
        x_train_pred = model.predict(x_train)
        train_mae_loss = np.mean(np.abs(x_train_pred - x_train), axis=1)

        # Get reconstruction loss threshold.
        threshold = np.max(train_mae_loss)
    #     print("Reconstruction error threshold: ", threshold)

        # Get test MAE loss.
        x_test_pred = model.predict(x_test)
        test_mae_loss = np.mean(np.abs(x_test_pred - x_test), axis=1)
        test_mae_loss = test_mae_loss.reshape((-1))

        # Detect all the samples which are anomalies.
        anomalies = test_mae_loss > threshold
    #     print("Number of anomaly samples: ", np.sum(anomalies))
    #     print("Indices of anomaly samples: ", np.where(anomalies))

        self.x_train = x_train
        self.x_test = x_test
        self.x_train_pred = x_train_pred
        self.train_mae_loss = train_mae_loss
        self.threshold = threshold
        self.x_test_pred = x_test_pred
        self.test_mae_loss = test_mae_loss
        self.anomalies = anomalies

        return anomalies

    def plot(self, plot_title='', save_folder='', ):
        '''
        (1) Plot MAE loss
        (2) Compare recontruction - Test with training data.
            Checking how the first sequence is learnt
        '''
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
        fig.suptitle(
            f'{plot_title}, MAE loss and Compare recontruction')

        ax1.hist(self.train_mae_loss, bins=50)
        ax1.set_title('Train MAE loss')
        ax1.set(xlabel='Train MAE loss', ylabel='No of samples')

        ax2.hist(self.test_mae_loss, bins=50)
        ax2.set_title("Test MAE loss")
        ax2.set(xlabel='Test MAE loss', ylabel='No of samples')

        ax3.set_title("Compare recontruction")
        ax3.plot(self.x_train[0], label='origin')
        ax3.plot(self.x_train_pred[0], label='predicted')
#         ax3.axis('equal')

        path = f'{save_folder}/{plot_title}.png'
        plt.savefig(path, facecolor='white', transparent=False)
#         plt.show()
        plt.close()

### `def findAnomaliesRange(anomalies, _train):`

In [12]:

def findAnomaliesRange(anomalies, autoperiod, _train):
    '''https://stackoverflow.com/questions/2361945/detecting-consecutive-integers-in-a-list
    '''
    _a = np.where(anomalies)[0]+len(_train)
    nums = sorted(set(_a))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s+autoperiod/2 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))


def findAnomaliesRange_v2(anomalies, _train):
    '''https://stackoverflow.com/questions/10016802/grouping-a-list-of-integers-with-nearest-values
    '''
    _a = np.where(anomalies)[0]+len(_train)
    _a.sort()
    diff = [_a[i+1]-_a[i] for i in range(len(_a)-1)]
    avg = sum(diff) / len(diff)
    anomalies_group = [[_a[0]]]
    for x in _a[1:]:
        if x - anomalies_group[-1][-1] < avg:
            anomalies_group[-1].append(x)
        else:
            anomalies_group.append([x])
    return anomalies_group


### `def plotAnomalies(df_test, anomalies):`

In [13]:
def plotAnomalies(df: pd.DataFrame,
                  anomalies: np.array,
                  plot_title: str = '',
                  save_folder=''
                  ):
    df_subset = df.iloc[np.where(anomalies)]
    print('len(df)=', len(df))
#     display(df_subset)

    N, M = 25, 6
    fig, ax = plt.subplots(figsize=(N, M))
    ax.set_title(plot_title)
    df.plot(legend=False, ax=ax)
    df_subset.plot(legend=False, ax=ax, color="r")

#     plt.axvline(int(ans["anomaly_start"][j])-100, color="r")
#     plt.axvline(int(ans["anomaly_end"][j])+100, color="r")

    path = f'{save_folder}/{plot_title}.png'
    print('> plotAnomalies(), path=', path)
    plt.savefig(path, facecolor='white', transparent=False)
#     plt.show()
    plt.close()

# Main

## Load data

In [14]:
path = '../data-sets/KDD-Cup/data/'

loader = DataLoader(path)
loader.loadData()

data_df_list = loader.getDataInDfList()

..........................................................................................................................................................................................................................................................

In [15]:
name, data_df = data_df_list[33]

df_train = data_df[data_df['label']=='train']['values'].to_frame()
df_test = data_df[data_df['label']=='test']['values'].to_frame()

display(data_df)

print(name)
print('df_train.shape =', df_train.shape)
print('df_test.shape =', df_test.shape)

,values,label
0,76.458008,train
1,76.362195,train
2,77.233139,train
3,77.797305,train
4,77.612248,train
...,...,...
7649,45.042070,test
7650,46.081865,test
7651,45.737770,test
7652,45.362732,test


034_UCR_Anomaly_1500.txt
df_train.shape = (1500, 1)
df_test.shape = (6154, 1)


## Build, Train and Test Autoencoder for all data

In [16]:
result_plot_folder = 'results'

if not os.path.exists(result_plot_folder):
    os.makedirs(result_plot_folder)

In [ ]:
for i, (name, data_df) in enumerate(data_df_list[25:]):
    print(f'=== {name} ======================================================')

    # Get data
    _train_df = data_df[data_df['label'] == 'train']['values'].to_frame()
    _test_df = data_df[data_df['label'] == 'test']['values'].to_frame()

    # Normalize data
    _train_df_value = normalize(_train_df)
    _test_df_value = normalize(_test_df, _train_df)

#     # Visualization
#     # Plot of train data (without anomaly)
#     plotTimeSeries(_train_df_value, title=f'{name}, Training ')
#     # Plot of test data (with anomaly)
#     plotTimeSeries(_train_df_value, 100, title=f'{name}, Testing')

    # Find autoperiod
    autoperiod = findAutoPeriod(_train_df['values'])
    autoperiod = int(autoperiod)
#     autoperiod += autoperiod % 2
    print('> autoperiod:', autoperiod)

    # Define Window Size
#     time_steps_ls = [autoperiod-int(autoperiod/2.6), autoperiod+int(autoperiod/2.6), autoperiod]
    time_steps_ls = [autoperiod]

    for time_steps in time_steps_ls:
        try:
            time_steps += time_steps % 2
            print('=== Window Size =', time_steps)

            # Discretize the normalized time series data
            _train_X = createSequences(_train_df_value.values, time_steps)
            _test_X = createSequences(_test_df_value.values, time_steps)
            print("> Training input shape: ", _train_X.shape)

            # Build model
            model_name = f'{name[:-4]}-{time_steps}windowsize-{autoperiod}autoperiod'

            model = buildAutoencoder(shape=(_train_X.shape[1], _train_X.shape[2]))
            # f'{time_steps}windowsize_{autoperiod}autoperiod'
            model._name = model_name
            model.summary()

            # Train model
            batch_size = int(time_steps/5)
            batch_size += batch_size % 2
            print('> batch_size =', batch_size)
            model, history = trainAutoencoder(
                _train_X, model, batch_size=batch_size)

            # Save the trained model
            saveAutoencoder(model, model_name)

            # Test model, detect anomaly
            detector = detectAnomalies(model)
            anomalies = detector.detect(_train_X, _test_X)
            # Save anomalies
            _save = np.where(anomalies)[0]+len(_train_X)
            np.savetxt(f'{result_plot_folder}/anomaly-{model_name}.txt',
                       _save,
                       delimiter=',')

            # Plot Autoencoder result
            detector.plot(save_folder=result_plot_folder,
                          plot_title=f'{model_name}-autoencoder')
            anomalies_ranges = findAnomaliesRange(anomalies, autoperiod, _train_X)

            print("> Reconstruction error threshold: ", detector.threshold)
            print("> Number of anomaly samples: ", np.sum(anomalies))
    #         print("> Indices of anomaly samples: ", np.where(anomalies))
            for _range in anomalies_ranges:
                print("> Indices of anomaly samples: ", np.where(_range))

            # Plot anomaly position
            plotAnomalies(df=_test_df,
                          anomalies=anomalies,
                          plot_title=f'anomaly-{model_name}__{anomalies_ranges}',
                          save_folder=result_plot_folder,
                          )

            keras.backend.clear_session()
            
        except:
            pass
        print()

#         break
#     break

=== 026_UCR_Anomaly_1700.txt ======================================================
> Normalizing by same `mean` `std`.
> Normalizing by other `mean` `std`.
[170.0, 85.0, 89.0, 44.0, 1700.0]
> autoperiod: 170
=== Window Size = 170
> Training input shape:  (1531, 170, 1)
> buildAutoencoder(), shape= (170, 1)

=== 027_UCR_Anomaly_1200.txt ======================================================
> Normalizing by same `mean` `std`.
> Normalizing by other `mean` `std`.
[171.0, 200.0, 92.0, 1200.0, 240.0]
> autoperiod: 171
=== Window Size = 172
> Training input shape:  (1029, 172, 1)
> buildAutoencoder(), shape= (172, 1)
Model: "027_UCR_Anomaly_1200-172windowsize-171autoperiod"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 86, 16)            80        
_________________________________________________________________
dropout_2 (Dropout)          (None, 86, 16)            0 

Epoch 2/50
63/63 - 1s - loss: 0.0158 - val_loss: 0.0576
Epoch 3/50
63/63 - 1s - loss: 0.0114 - val_loss: 0.0615
Epoch 4/50
63/63 - 2s - loss: 0.0099 - val_loss: 0.0535
Epoch 5/50
63/63 - 1s - loss: 0.0091 - val_loss: 0.0631
Epoch 6/50
63/63 - 1s - loss: 0.0084 - val_loss: 0.0477
> Saved `models/autoencoder/031_UCR_Anomaly_2700-180windowsize-180autoperiod.h5` !
> Test input shape:  (4621, 180, 1)
> Reconstruction error threshold:  0.22568458893991827
> Number of anomaly samples:  51
> Indices of anomaly samples:  (array([0, 1]),)
len(df)= 4800
> plotAnomalies(), path= results/anomaly-031_UCR_Anomaly_2700-180windowsize-180autoperiod__[(5440, 5490)].png

=== 032_UCR_Anomaly_1000.txt ======================================================
> Normalizing by same `mean` `std`.
> Normalizing by other `mean` `std`.
[167.0, 200.0, 91.0, 143.0, 1000.0]
> autoperiod: 167
=== Window Size = 168
> Training input shape:  (833, 168, 1)
> buildAutoencoder(), shape= (168, 1)
Model: "032_UCR_Anomaly_1000-1

Epoch 1/50
101/101 - 2s - loss: 0.1229 - val_loss: 0.0315
Epoch 2/50
101/101 - 3s - loss: 0.0169 - val_loss: 0.0300
Epoch 3/50
101/101 - 2s - loss: 0.0112 - val_loss: 0.0212
Epoch 4/50
101/101 - 1s - loss: 0.0087 - val_loss: 0.0232
Epoch 5/50
101/101 - 2s - loss: 0.0075 - val_loss: 0.0170
Epoch 6/50
101/101 - 2s - loss: 0.0066 - val_loss: 0.0233
Epoch 7/50
101/101 - 2s - loss: 0.0060 - val_loss: 0.0186
Epoch 8/50
101/101 - 1s - loss: 0.0056 - val_loss: 0.0143
Epoch 9/50
101/101 - 1s - loss: 0.0054 - val_loss: 0.0233
Epoch 10/50
101/101 - 1s - loss: 0.0052 - val_loss: 0.0203
Epoch 11/50
101/101 - 1s - loss: 0.0047 - val_loss: 0.0162
Epoch 12/50
101/101 - 1s - loss: 0.0045 - val_loss: 0.0196
Epoch 13/50
101/101 - 1s - loss: 0.0044 - val_loss: 0.0194
> Saved `models/autoencoder/036_UCR_Anomaly_4200-176windowsize-175autoperiod.h5` !
> Test input shape:  (3126, 176, 1)
> Reconstruction error threshold:  0.16033809872148994
> Number of anomaly samples:  0
len(df)= 3301
> plotAnomalies(), pat

559/559 - 4s - loss: 0.0982 - val_loss: 0.0694
Epoch 2/50
559/559 - 2s - loss: 0.0499 - val_loss: 0.0528
Epoch 3/50
559/559 - 1s - loss: 0.0417 - val_loss: 0.0566
Epoch 4/50
559/559 - 1s - loss: 0.0368 - val_loss: 0.0652
Epoch 5/50
559/559 - 1s - loss: 0.0351 - val_loss: 0.0778
Epoch 6/50
559/559 - 1s - loss: 0.0338 - val_loss: 0.0941
Epoch 7/50
559/559 - 1s - loss: 0.0341 - val_loss: 0.1163
> Saved `models/autoencoder/039_UCR_Anomaly_5000-36windowsize-35autoperiod.h5` !
> Test input shape:  (24915, 36, 1)
> Reconstruction error threshold:  0.42967171484816746
> Number of anomaly samples:  72
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of ano

784/784 - 8s - loss: 0.0980 - val_loss: 0.0597
Epoch 2/50
784/784 - 4s - loss: 0.0538 - val_loss: 0.0633
Epoch 3/50
784/784 - 4s - loss: 0.0496 - val_loss: 0.0752
Epoch 4/50
784/784 - 4s - loss: 0.0474 - val_loss: 0.0836
Epoch 5/50
784/784 - 4s - loss: 0.0445 - val_loss: 0.0609
Epoch 6/50
784/784 - 4s - loss: 0.0420 - val_loss: 0.0423
Epoch 7/50
784/784 - 6s - loss: 0.0419 - val_loss: 0.0587
Epoch 8/50
784/784 - 8s - loss: 0.0413 - val_loss: 0.0478
Epoch 9/50
784/784 - 5s - loss: 0.0381 - val_loss: 0.1017
Epoch 10/50
784/784 - 3s - loss: 0.0378 - val_loss: 0.0754
Epoch 11/50
784/784 - 2s - loss: 0.0384 - val_loss: 0.0638
> Saved `models/autoencoder/042_UCR_Anomaly_7000-36windowsize-35autoperiod.h5` !
> Test input shape:  (22824, 36, 1)
> Reconstruction error threshold:  0.31504504085627516
> Number of anomaly samples:  3
> Indices of anomaly samples:  (array([0, 1]),)
> Indices of anomaly samples:  (array([0, 1]),)
len(df)= 22859
> plotAnomalies(), path= results/anomaly-042_UCR_Anomaly

...

Epoch 1/50
153/153 - 2s - loss: 0.0777 - val_loss: 0.0751
Epoch 2/50
153/153 - 1s - loss: 0.0150 - val_loss: 0.0483
Epoch 3/50
153/153 - 1s - loss: 0.0119 - val_loss: 0.0394
Epoch 4/50
153/153 - 1s - loss: 0.0108 - val_loss: 0.0569
Epoch 5/50
153/153 - 1s - loss: 0.0089 - val_loss: 0.0565
Epoch 6/50
153/153 - 1s - loss: 0.0079 - val_loss: 0.0539
Epoch 7/50
153/153 - 1s - loss: 0.0075 - val_loss: 0.0698
Epoch 8/50
153/153 - 1s - loss: 0.0066 - val_loss: 0.0718
> Saved `models/autoencoder/052_UCR_Anomaly_3500-100windowsize-100autoperiod.h5` !
> Test input shape:  (7709, 100, 1)
> Reconstruction error threshold:  0.22303227717349108
> Number of anomaly samples:  0
len(df)= 7808
> plotAnomalies(), path= results/anomaly-052_UCR_Anomaly_3500-100windowsize-100autoperiod__[].png

=== 053_UCR_Anomaly_1500.txt ======================================================
> Normalizing by same `mean` `std`.
> Normalizing by other `mean` `std`.
[58.0, 56.0, 54.0, 60.0, 12.0]
> autoperiod: 58
=== Window S

Model: "058_UCR_Anomaly_10000-100windowsize-100autoperiod"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 50, 16)            80        
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 25, 8)             520       
_________________________________________________________________
conv1d_transpose_3 (Conv1DTr (None, 50, 8)             264       
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 8)             0         
_________________________________________________________________
conv1d_transpose_4 (Conv1DTr (None, 100, 16)           528       
_________________________________________________________________
conv1d_transpose_

446/446 - 2s - loss: 0.0806 - val_loss: 0.0489
Epoch 2/50
446/446 - 1s - loss: 0.0284 - val_loss: 0.0256
Epoch 3/50
446/446 - 1s - loss: 0.0222 - val_loss: 0.0175
Epoch 4/50
446/446 - 1s - loss: 0.0210 - val_loss: 0.0191
Epoch 5/50
446/446 - 1s - loss: 0.0193 - val_loss: 0.0195
Epoch 6/50
446/446 - 1s - loss: 0.0180 - val_loss: 0.0206
Epoch 7/50
446/446 - 1s - loss: 0.0179 - val_loss: 0.0159
Epoch 8/50
446/446 - 1s - loss: 0.0180 - val_loss: 0.0265
Epoch 9/50
446/446 - 2s - loss: 0.0161 - val_loss: 0.0115
Epoch 10/50
446/446 - 2s - loss: 0.0162 - val_loss: 0.0133
Epoch 11/50
446/446 - 2s - loss: 0.0156 - val_loss: 0.0162
Epoch 12/50
446/446 - 1s - loss: 0.0144 - val_loss: 0.0190
Epoch 13/50
446/446 - 1s - loss: 0.0140 - val_loss: 0.0160
Epoch 14/50
446/446 - 1s - loss: 0.0159 - val_loss: 0.0249
> Saved `models/autoencoder/065_UCR_Anomaly_3000-28windowsize-27autoperiod.h5` !
> Test input shape:  (6974, 28, 1)
> Reconstruction error threshold:  0.22080710482604338
> Number of anomaly sam

> Normalizing by same `mean` `std`.
> Normalizing by other `mean` `std`.
[115.0, 118.0, 231.0, 79.0, 76.0]
> autoperiod: 231
=== Window Size = 232
> Training input shape:  (17324, 232, 1)
> buildAutoencoder(), shape= (232, 1)
Model: "070_UCR_Anomaly_17555-232windowsize-231autoperiod"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 116, 16)           80        
_________________________________________________________________
dropout_2 (Dropout)          (None, 116, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 58, 8)             520       
_________________________________________________________________
conv1d_transpose_3 (Conv1DTr (None, 116, 8)            264       
_________________________________________________________________
dropout_3 (Dropout)          (None, 116, 8)            

Epoch 1/50


In [ ]:
# model = keras.models.load_model('models/autoencoder/001_UCR_Anomaly_35000-132_windowsize-212_autoperiod.h5')

# detector = detectAnomalies(model)
# anomalies = detector.detect(_train_X, _test_X)
# detector.plot(save_folder = result_plot_folder,
#               plot_title = model_name)


In [ ]:
# anomalies_ranges = findAnomaliesRange(anomalies, autoperiod, _train_X)
# anomalies_ranges
# [b-a for a,b in anomalies_ranges]

In [ ]:
# _save = np.where(anomalies)[0]+len(_train_X)

# print(_save)
# np.savetxt('test.out', _save, delimiter=',')   # X is an array

In [ ]:
# np.loadtxt("test.out")

In [ ]:
keras.backend.clear_session()
del model

In [ ]:
keras.__version__